In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
import sklearn.utils
from interpret import show
from sklearn.metrics import f1_score, accuracy_score
import pandas_profiling as pp
from sklearn.cluster import KMeans 
import warnings
from interpret.glassbox import (LogisticRegression,
                                ClassificationTree, 
                                ExplainableBoostingClassifier)
warnings.filterwarnings('ignore')
logisticRegr = LogisticRegression()
from sklearn.svm import SVC
%matplotlib inline

In [58]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [59]:
df.head

<bound method NDFrame.head of      PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
8688     9276_01     Europa     False    A/98/P    55 Cancri e  41.0   True   
8689     9278_01      Earth      True  G/1499/S  PSO J318.5-22  18.0  False   
8690     9279_01      Earth     False  G/1500/S    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa     False   E/608/S    55 Cancri e  32.0  False   
8692     9280_02     Europa     False   E/608/S    TRAPPIST-1e  44.0  False   

      RoomService  Fo

In [60]:
df.drop(['Name',"PassengerId","Cabin"],axis=1,inplace=True)
df_test.drop(['Name',"Cabin"],axis=1,inplace=True)
df.head

<bound method NDFrame.head of      HomePlanet CryoSleep    Destination   Age    VIP  RoomService  FoodCourt  \
0        Europa     False    TRAPPIST-1e  39.0  False          0.0        0.0   
1         Earth     False    TRAPPIST-1e  24.0  False        109.0        9.0   
2        Europa     False    TRAPPIST-1e  58.0   True         43.0     3576.0   
3        Europa     False    TRAPPIST-1e  33.0  False          0.0     1283.0   
4         Earth     False    TRAPPIST-1e  16.0  False        303.0       70.0   
...         ...       ...            ...   ...    ...          ...        ...   
8688     Europa     False    55 Cancri e  41.0   True          0.0     6819.0   
8689      Earth      True  PSO J318.5-22  18.0  False          0.0        0.0   
8690      Earth     False    TRAPPIST-1e  26.0  False          0.0        0.0   
8691     Europa     False    55 Cancri e  32.0  False          0.0     1049.0   
8692     Europa     False    TRAPPIST-1e  44.0  False        126.0     4688.0  

In [61]:
#pp.ProfileReport(df_test, title = 'Pandas Profiling report of "Train" set', html = {'style':{'full_width': True}})

In [62]:
df_test["RoomService"].fillna(df_test.RoomService.mean, inplace=True)
df_test["FoodCourt"].fillna(df_test.FoodCourt.mean, inplace=True)
df_test["ShoppingMall"].fillna(df_test.ShoppingMall.mean, inplace=True)
df_test["Spa"].fillna(df_test.Spa.mean, inplace=True)
df_test["VRDeck"].fillna(df_test.VRDeck.mean, inplace=True)

In [63]:
#pp.ProfileReport(df_test, title = 'Pandas Profiling report of "Train" set', html = {'style':{'full_width': True}})

In [64]:
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df_test_pass=df_test["PassengerId"]
df_test.drop(["PassengerId"],axis=1,inplace=True)
df.shape

(6923, 11)

In [65]:
#pp.ProfileReport(df, title = 'Pandas Profiling report of "Train" set', html = {'style':{'full_width': True}})

In [66]:
encode_columns = ['HomePlanet','CryoSleep','Destination','VIP','Transported']
for i in encode_columns:
    df[i] = df[i].astype('category')
    df[i] = df[i].cat.codes

encode_columns = ['HomePlanet','CryoSleep','Destination','VIP']
for i in encode_columns:
    df_test[i] = df_test[i].astype('category')
    df_test[i] = df_test[i].cat.codes

df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1


In [67]:
X=df.drop(["Transported"],axis=1)
Y=df['Transported']
X_train = X.iloc[:5000,:]
X_dev = X.iloc[5000:,:]
Y_train = Y.iloc[:5000]
Y_dev = Y.iloc[5000:]
X_test=df_test

In [68]:
lr = LogisticRegression(random_state=2021, feature_names=X_train.columns, penalty='l1', solver='liblinear')
lr.fit(X_train, Y_train)

In [69]:
Y_pred = lr.predict(X_dev)
print(f"F1 Score {f1_score(Y_dev, Y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(Y_dev, Y_pred)}")

F1 Score 0.7964020862007853
Accuracy 0.796671866874675


In [70]:
# %% Explain global logistic regression model
lr_global = lr.explain_global(name='Logistic Regression')
show(lr_global)

<!-- http://127.0.0.1:7001/140446350832352/ -->

In [71]:
# %% Fit Explainable Boosting Machine
ebm = ExplainableBoostingClassifier(random_state=2021)
ebm.fit(X_train, Y_train) 
print("Training finished.")
Y_pred = ebm.predict(X_dev)
print(f"F1 Score {f1_score(Y_dev, Y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(Y_dev, Y_pred)}")

# %% Explain locally
ebm_local = ebm.explain_local(X_dev[:100], Y_dev[:100], name='EBM')
show(ebm_local)



Training finished.
F1 Score 0.8013770656594861
Accuracy 0.8018720748829953


<!-- http://127.0.0.1:7001/140446346388240/ -->

In [72]:
# %% Explain globally
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7001/140447074241888/ -->

In [73]:
Y_test = ebm.predict(X_test)
Y_test.shape

ValueError: Missing values are currently not supported.

In [ ]:
df_test=pd.read_csv('test.csv')
submission = pd.DataFrame({
        "PassengerId": df_test_pass,
        "Transported": Y_test
    })
submission.replace(1,'True', inplace=True)
submission.replace(0,'False', inplace=True)

In [ ]:
submission.to_csv('submission.csv', index=False)